In [1]:
from graphframes import *

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc


 

In [3]:
import os
from pyspark.sql import SparkSession
os.environ['PYSPARK_PYTHON'] = "/usr/bin/python3"
spark = SparkSession.builder.appName("graph processing")\
.config("spark.jars.packages","graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
.master("local[*]").getOrCreate()

25/04/12 13:20:31 WARN Utils: Your hostname, bigdata2024 resolves to a loopback address: 127.0.1.1; using 10.3.132.222 instead (on interface ens3)
25/04/12 13:20:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hadoop/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e628feeb-d10e-493a-9d84-875a794ccb3f;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 149ms :: artifacts dl 6ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------

In [4]:
v= spark.createDataFrame([('1', 'Carter', 'Derrick', 50), 
                                  ('2', 'May', 'Derrick', 26),
                                 ('3', 'Mills', 'Jeff', 80),
                                  ('4', 'Hood', 'Robert', 65),
                                  ('5', 'Banks', 'Mike', 93),
                                 ('98', 'Berg', 'Tim', 28),
                                 ('99', 'Page', 'Allan', 16)],
                                 ['id', 'name', 'firstname', 'age'])#create edges dataframe

In [5]:
e = spark.createDataFrame([('1', '2', 'friend'), 
                               ('2', '1', 'friend'),
                              ('3', '1', 'friend'),
                              ('1', '3', 'friend'),
                               ('2', '3', 'follows'),
                               ('3', '4', 'friend'),
                               ('4', '3', 'friend'),
                               ('5', '3', 'friend'),
                               ('3', '5', 'friend'),
                               ('4', '5', 'follows'),
                              ('98', '99', 'friend'),
                              ('99', '98', 'friend')],
                              ['src', 'dst', 'type'])

In [6]:
g = GraphFrame(v, e)

/home/hadoop/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [7]:
g.inDegrees.show()

/home/hadoop/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")
[Stage 0:>                                                          (0 + 4) / 4]

+---+--------+
| id|inDegree|
+---+--------+
|  1|       2|
|  2|       1|
|  3|       4|
|  4|       1|
|  5|       2|
| 98|       1|
| 99|       1|
+---+--------+



In [8]:
g.edges.filter("type = 'follows'").count()

2

In [9]:
 g.vertices.show()

+---+------+---------+---+
| id|  name|firstname|age|
+---+------+---------+---+
|  1|Carter|  Derrick| 50|
|  2|   May|  Derrick| 26|
|  3| Mills|     Jeff| 80|
|  4|  Hood|   Robert| 65|
|  5| Banks|     Mike| 93|
| 98|  Berg|      Tim| 28|
| 99|  Page|    Allan| 16|
+---+------+---------+---+



In [10]:
g.edges.show()

+---+---+-------+
|src|dst|   type|
+---+---+-------+
|  1|  2| friend|
|  2|  1| friend|
|  3|  1| friend|
|  1|  3| friend|
|  2|  3|follows|
|  3|  4| friend|
|  4|  3| friend|
|  5|  3| friend|
|  3|  5| friend|
|  4|  5|follows|
| 98| 99| friend|
| 99| 98| friend|
+---+---+-------+



In [11]:
inDegreeDF=g.inDegrees
outDegreeDF=g.outDegrees
degreeDF=g.degrees

inDegreeDF.sort(['inDegree'],ascending=[0]).show() # Sort and show

+---+--------+
| id|inDegree|
+---+--------+
|  3|       4|
|  1|       2|
|  5|       2|
|  2|       1|
|  4|       1|
| 98|       1|
| 99|       1|
+---+--------+



In [12]:
outDegreeDF.sort(['outDegree'],ascending=[0]).show()

+---+---------+
| id|outDegree|
+---+---------+
|  3|        3|
|  1|        2|
|  2|        2|
|  4|        2|
|  5|        1|
| 98|        1|
| 99|        1|
+---+---------+



In [13]:
results = g.pageRank(resetProbability=0.01, maxIter=20)

In [14]:
results3 = g.pageRank(resetProbability=0.15, maxIter=10, sourceId="1")

In [15]:
results.vertices.select("id", "pagerank").show()

+---+-------------------+
| id|           pagerank|
+---+-------------------+
|  3| 1.9926565142372492|
| 98| 1.0000000000000004|
| 99| 1.0000000000000004|
|  5| 0.9980278491870183|
|  1|  0.890802837490737|
|  4|   0.66757817627641|
|  2|0.45093462280858443|
+---+-------------------+



In [16]:
results3.vertices.select("id", "pagerank").show()

+---+-------------------+
| id|           pagerank|
+---+-------------------+
|  3|0.33891806394017526|
| 98|                0.0|
| 99|                0.0|
|  5|0.13689847609530284|
|  1|0.30079283470815427|
|  4|0.09616868544059574|
|  2| 0.1272219398157718|
+---+-------------------+



In [17]:
results = g.pageRank(resetProbability=0.15, tol=0.01)

In [18]:
results.vertices.select("id", "pagerank").show()

+---+------------------+
| id|          pagerank|
+---+------------------+
|  3| 1.853919642738813|
| 98|1.0225331112091938|
| 99|1.0225331112091938|
|  5|0.9703579134677663|
|  1|0.9055074972891308|
|  4|0.6873519241384106|
|  2|0.5377967999474921|
+---+------------------+



In [19]:
from graphframes.examples import Graphs

In [20]:
g = Graphs(sqlContext).friends()  # Get example graph

NameError: name 'sqlContext' is not defined

In [ ]:
g.edges.show()

In [ ]:
g.vertices.show()

In [ ]:
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")

In [ ]:
motifs.show()

In [ ]:
motifs.filter("b.age > 30").show()

In [ ]:
mutualFriends = g.find("(a)-[]->(b); (b)-[]->(c); (c)-[]->(b); (b)-[]->(a)").dropDuplicates()

In [ ]:
mutualFriends.filter('a.id == 2 and c.id == 3').show()